In [2]:
#pip install shap


  Obtaining dependency information for shap from https://files.pythonhosted.org/packages/72/28/d4dcf1b06761c916f642e0fe1293ed8081ce32991b32eada2212f639433c/shap-0.47.1-cp311-cp311-win_amd64.whl.metadata
  Obtaining dependency information for slicer==0.0.8 from https://files.pythonhosted.org/packages/63/81/9ef641ff4e12cbcca30e54e72fb0951a2ba195d0cda0ba4100e532d929db/slicer-0.0.8-py3-none-any.whl.metadata
   ---------------------------------------- 0.0/489.8 kB ? eta -:--:--
   ----- ---------------------------------- 61.4/489.8 kB 3.2 MB/s eta 0:00:01
   ----- ---------------------------------- 61.4/489.8 kB 3.2 MB/s eta 0:00:01
   ----- ---------------------------------- 61.4/489.8 kB 3.2 MB/s eta 0:00:01
   ----- ---------------------------------- 61.4/489.8 kB 3.2 MB/s eta 0:00:01
   ----- ---------------------------------- 61.4/489.8 kB 3.2 MB/s eta 0:00:01
   ----- ---------------------------------- 61.4/489.8 kB 3.2 MB/s eta 0:00:01
   ----- ---------------------------------- 61.

In [4]:
import numpy as np
import pandas as pd

from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras import regularizers
from tensorflow.keras.callbacks import EarlyStopping

import shap
import warnings
import logging

warnings.filterwarnings("ignore")
logger = logging.getLogger('shap')
logger.disabled = True

### Load the dataset

In [10]:
credit_card_df = pd.read_csv(r"C:\Users\Zaya\OneDrive - Eotvos Lorand Tudomanyegyetem\Dokumentumok\2023 ELTE Study\DSLAB2\data\creditcard.csv")

In [11]:
ae_df = pd.read_csv(r"C:\Users\Zaya\OneDrive - Eotvos Lorand Tudomanyegyetem\Dokumentumok\2023 ELTE Study\DSLAB2\data\prefect_autoencoder_data_top_10k.csv")

In [12]:
kdd_df = pd.read_csv(r"C:\Users\Zaya\OneDrive - Eotvos Lorand Tudomanyegyetem\Dokumentumok\2023 ELTE Study\DSLAB2\data\kddcup.newtestdata_10_percent_unlabeled.csv")

In [13]:
df = credit_card_df

In [14]:
print("Dataset Shape:", df.shape)

df.head()

Dataset Shape: (284807, 31)


,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0


In [15]:
df = df.drop(['Time'], axis=1)

### Data Normalization

In [16]:
for col in df.columns[:-1]:
    min_val = df[col].min()
    max_val = df[col].max()
    if min_val != max_val:
        df[col] = (df[col] - min_val) / (max_val - min_val)

In [17]:
df.head()

,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.935192,0.766490,0.881365,0.313023,0.763439,0.267669,0.266815,0.786444,0.475312,0.510600,...,0.561184,0.522992,0.663793,0.391253,0.585122,0.394557,0.418976,0.312697,0.005824,0
1,0.978542,0.770067,0.840298,0.271796,0.766120,0.262192,0.264875,0.786298,0.453981,0.505267,...,0.557840,0.480237,0.666938,0.336440,0.587290,0.446013,0.416345,0.313423,0.000105,0
2,0.935217,0.753118,0.868141,0.268766,0.762329,0.281122,0.270177,0.788042,0.410603,0.513018,...,0.565477,0.546030,0.678939,0.289354,0.559515,0.402727,0.415489,0.311911,0.014739,0
3,0.941878,0.765304,0.868484,0.213661,0.765647,0.275559,0.266803,0.789434,0.414999,0.507585,...,0.559734,0.510277,0.662607,0.223826,0.614245,0.389197,0.417669,0.314371,0.004807,0
4,0.938617,0.776520,0.864251,0.269796,0.762975,0.263984,0.268968,0.782484,0.490950,0.524303,...,0.561327,0.547271,0.663392,0.401270,0.566343,0.507497,0.420561,0.317490,0.002724,0


### Train and Test Split

In [18]:
X = df.iloc[:,:-1]
y = df.iloc[:, -1]

print('x shape:', X.shape)
print('y shape:', y.shape)
print(y.value_counts())

train_idx = y[y==0].index.values
test_idx = y[y==1].index.values

X_train = X.iloc[train_idx]
y_train = y[train_idx]

X_test = X.iloc[test_idx]
y_test = y[test_idx]

x shape: (284807, 29)
y shape: (284807,)
0    284315
1       492
Name: Class, dtype: int64


### Initializing Class and Parameters

In [19]:
class ExplainAnomaliesUsingSHAP:
    autoencoder = None
    num_anomalies_to_explain = None
    reconstruction_error_percent = None
    shap_values_selection = None
    counter = None

    def __init__(self, num_anomalies_to_explain=100, reconstruction_error_percent=0.5, shap_values_selection='mean'):
        self.num_anomalies_to_explain = num_anomalies_to_explain
        self.reconstruction_error_percent = reconstruction_error_percent
        self.shap_values_selection = shap_values_selection

### Training the Autoencoder

In [22]:
input_dim = X_train.shape[1]

input_layer = Input(shape=(input_dim,))

encoder = Dense(int(input_dim / 2), activation="relu", activity_regularizer=regularizers.l1(10e-7))(input_layer)
encoder = Dense(int(input_dim / 4), activation="relu", kernel_regularizer=regularizers.l2(10e-7))(encoder)

decoder = Dense(int(input_dim / 2), activation='relu', kernel_regularizer=regularizers.l2(10e-7))(encoder)
decoder = Dense(input_dim, activation='sigmoid', kernel_regularizer=regularizers.l2(10e-7))(decoder)

autoencoder = Model(inputs=input_layer, outputs=decoder)

autoencoder.summary()

autoencoder.compile(optimizer='adam', loss='mean_squared_error', metrics=['mse'])

earlystopper = EarlyStopping(monitor='val_loss', patience=5, verbose=1)
autoencoder.fit(X_train, X_train, epochs=1000, batch_size=64, shuffle=True,
                 validation_split=0.1, verbose=2, callbacks=[earlystopper])

Model: "functional_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)      │ (None, 29)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 14)             │           420 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 7)              │           105 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ (None, 14)             │           112 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 29)             │           435 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,072 (4.19 KB)

 Trainable params: 1,072 (4.19 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/1000
3999/3999 - 9s - 2ms/step - loss: 0.0021 - mse: 0.0019 - val_loss: 0.0010 - val_mse: 9.4710e-04
Epoch 2/1000
3999/3999 - 7s - 2ms/step - loss: 9.7827e-04 - mse: 8.9181e-04 - val_loss: 8.3991e-04 - val_mse: 7.4617e-04
Epoch 3/1000
3999/3999 - 7s - 2ms/step - loss: 8.4116e-04 - mse: 7.5174e-04 - val_loss: 8.0691e-04 - val_mse: 7.2118e-04
Epoch 4/1000
3999/3999 - 8s - 2ms/step - loss: 8.2463e-04 - mse: 7.4080e-04 - val_loss: 7.9849e-04 - val_mse: 7.1696e-04
Epoch 5/1000
3999/3999 - 8s - 2ms/step - loss: 8.1608e-04 - mse: 7.3615e-04 - val_loss: 7.7830e-04 - val_mse: 7.0035e-04
Epoch 6/1000
3999/3999 - 9s - 2ms/step - loss: 8.1052e-04 - mse: 7.3297e-04 - val_loss: 7.9010e-04 - val_mse: 7.1326e-04
Epoch 7/1000
3999/3999 - 8s - 2ms/step - loss: 8.0797e-04 - mse: 7.3106e-04 - val_loss: 7.8388e-04 - val_mse: 7.0787e-04
Epoch 8/1000
3999/3999 - 10s - 2ms/step - loss: 8.0616e-04 - mse: 7.2951e-04 - val_loss: 7.8233e-04 - val_mse: 7.0566e-04
Epoch 9/1000
3999/3999 - 8s - 2ms/step - lo

In [100]:
print(X_test)
predictions = autoencoder.predict(X_test)
print(predictions)

              V1        V2        V3        V4        V5        V6        V7  \
541     0.919012  0.787855  0.809517  0.429154  0.762201  0.248677  0.249897   
623     0.906588  0.733944  0.856275  0.353384  0.774870  0.252314  0.267339   
4920    0.919163  0.785821  0.831180  0.355228  0.760185  0.262258  0.268781   
6108    0.883578  0.781591  0.792483  0.370723  0.758122  0.245862  0.244056   
6329    0.979262  0.799121  0.762821  0.461731  0.790114  0.249369  0.275794   
...          ...       ...       ...       ...       ...       ...       ...   
279863  0.925541  0.779136  0.759117  0.329475  0.755171  0.242806  0.259977   
280143  0.981714  0.780863  0.750697  0.314516  0.768696  0.249683  0.256746   
280149  0.946807  0.779143  0.799053  0.272690  0.758173  0.262986  0.251741   
281144  0.905394  0.773440  0.743844  0.332480  0.760057  0.233425  0.251904   
281674  0.992135  0.768931  0.792646  0.270046  0.773466  0.262048  0.266714   

              V8        V9       V10  .

### Getting Top Anomalies

In [37]:
predictions = autoencoder.predict(X_test)
square_errors = np.power(X_test - predictions, 2)
mse_series = pd.Series(np.mean(square_errors, axis=1))

most_anomal_trx = mse_series.sort_values(ascending=False)
columns = ["id", "mse_all_columns"]
columns.extend(["squared_error_" + x for x in list(X_test.columns)])

items = []
for x in most_anomal_trx.iteritems():
    item = [x[0], x[1]]
    item.extend(square_errors.loc[x[0]])
    items.append(item)

df_anomalies = pd.DataFrame(items, columns=columns)
df_anomalies.set_index('id', inplace=True)

top_anomalies_to_explain = df_anomalies.head(100).index  # Heden anomaly explain hiihee songoh

16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 


In [38]:
df_anomalies.head(10)

,mse_all_columns,squared_error_V1,squared_error_V2,squared_error_V3,squared_error_V4,squared_error_V5,squared_error_V6,squared_error_V7,squared_error_V8,squared_error_V9,...,squared_error_V20,squared_error_V21,squared_error_V22,squared_error_V23,squared_error_V24,squared_error_V25,squared_error_V26,squared_error_V27,squared_error_V28,squared_error_Amount
id,,,,,,,,,,,,,,,,,,,,,
154587,0.075486,0.195796,0.075332,0.188821,0.072970,0.012399,0.002954,0.068608,0.165092,0.232141,...,0.014786,0.114831,0.159087,0.007808,6.062086e-03,0.006387,0.005611,0.010896,0.000173,1.265303e-05
154684,0.074428,0.190577,0.089571,0.191255,0.070806,0.013065,0.003069,0.076945,0.179673,0.231601,...,0.017366,0.128113,0.160666,0.010497,5.549781e-03,0.006109,0.003582,0.012214,0.000472,1.276060e-05
150644,0.066313,0.000002,0.021594,0.009773,0.013131,0.000138,0.002351,0.004445,0.002149,0.044984,...,0.000869,0.001981,0.000428,0.000723,3.735301e-03,0.000836,0.012705,0.000760,0.000014,3.983169e-07
42528,0.065162,0.000065,0.022407,0.003524,0.038788,0.000772,0.001020,0.004380,0.002819,0.071982,...,0.001329,0.002506,0.002319,0.000983,1.252069e-04,0.001042,0.003837,0.001100,0.000005,9.371168e-05
8296,0.064659,0.003056,0.030986,0.011821,0.020396,0.000110,0.002134,0.004426,0.002390,0.055270,...,0.001450,0.001778,0.000093,0.000649,1.763152e-02,0.012110,0.032150,0.000962,0.000003,1.271061e-05
150647,0.064485,0.000623,0.027854,0.006130,0.048191,0.000224,0.002066,0.006917,0.003470,0.039522,...,0.001290,0.002760,0.004690,0.001105,1.344941e-04,0.000097,0.009284,0.000549,0.000008,1.065260e-05
150601,0.063977,0.000232,0.020397,0.005402,0.001646,0.000169,0.003264,0.005276,0.002051,0.015432,...,0.001203,0.002279,0.002427,0.000671,1.224012e-04,0.002121,0.000530,0.001035,0.000012,1.242146e-05
154371,0.063859,0.169380,0.054939,0.177662,0.055777,0.011226,0.001247,0.053929,0.090465,0.161036,...,0.008374,0.071915,0.073084,0.003871,4.441127e-03,0.001299,0.008144,0.009039,0.000239,1.196016e-05
150665,0.063308,0.001887,0.022617,0.011494,0.028982,0.001217,0.001428,0.006262,0.004715,0.045310,...,0.000315,0.001944,0.000004,0.001307,2.604857e-08,0.001059,0.003351,0.000201,0.000187,2.095368e-05


In [41]:
df_anomalies.shape

(492, 30)

In [40]:
top_anomalies_to_explain

Int64Index([154587, 154684, 150644,  42528,   8296, 150647, 150601, 154371,
            150665, 150654,  42635, 150684,   8615,  41943,   9035,  42473,
             42609,  42590,  42549,   9252, 154234,   8335, 150662, 150663,
              9487,  42674, 150661, 150660,   9509, 150692,  10568,  42696,
             42741,   9179, 150668, 150669, 150666, 150667,  42700, 152295,
            150925,  42856,  42936, 153823, 153835,  42756,  42945,  42769,
             42958, 150697,  42784,  42887, 150687, 150715,  10801,  10897,
             43681, 150678, 150677,  10690,  10630, 151519,  43204,  43428,
             11343, 151008, 151009, 151006, 151007, 150679, 150680,  11710,
            152223, 151011, 151462,  11841,  11880,  44223,  12070,  43061,
             12108,  43160,  12261,  12369,  10891,  44270, 102443, 102445,
            102446, 102441, 102444, 102442,  17480,  14170,  17453,  17407,
             17366,  14197,  17317,  14211],
           dtype='int64', name='id')

### Errors Data Frame per Record

In [44]:
def get_errors_df_per_record(record):
    prediction = autoencoder.predict(np.array([[record]])[0])[0]
    square_errors = np.power(record - prediction, 2)
    errors_df = pd.DataFrame({'col_name': square_errors.index, 'err': square_errors}).reset_index(drop=True)
    total_mse = np.mean(square_errors)
    errors_df.sort_values(by='err', ascending=False, inplace=True)
    return errors_df, total_mse

### Calculating Features with Highest Reconstruction Error

In [45]:
def get_num_features_with_highest_reconstruction_error(total_squared_error, errors_df):
    error = 0
    for num_of_features, index in enumerate(errors_df.index):
        error += errors_df.loc[index, 'err']
        if error >= 0.5 * total_squared_error:  # 0.5 is the default reconstruction_error_percent
            break
    return num_of_features + 1

### Predicting Feature Value

In [153]:
def func_predict_feature(record):
    print("Record")
    print(record)
    record_prediction = autoencoder.predict(record)
    return record_prediction

### Creating Background Set for SHAP

In [57]:
background_set = X_train.head(200)  # Get first 200 records (adjust the number as needed)

### Getting SHAP Values for Features

In [154]:
def get_shap_values(df_top_err, record_to_explain, num_of_features): 
     
    shap_values_all_features = [[] for num in range(num_of_features)]  
    
    for i in range(num_of_features):
        counter = df_top_err.index[i]
        print("Orig Counter:", counter)
        explainer = shap.KernelExplainer(func_predict_feature, background_set)
        shap_values = explainer.shap_values(record_to_explain, nsamples='auto')
        print("Shap value")
        print(shap_values)
        shap_values_all_features[i] = shap_values
     
    shap_values_all_features = np.fabs(shap_values_all_features)

    return shap_values_all_features

In [155]:
def get_highest_shap_values(shap_values_df):

        all_explaining_features_df = pd.DataFrame()

        for i in range(shap_values_df.shape[0]):
            shap_values = shap_values_df.iloc[i]

            if self.shap_values_selection == 'mean':
                treshold_val = np.mean(shap_values)

            elif self.shap_values_selection == 'median':
                treshold_val = np.median(shap_values)

            elif self.shap_values_selection == 'constant':
                num_explaining_features = 5
                explaining_features = shap_values_df[i:i + 1].stack().nlargest(num_explaining_features)
                all_explaining_features_df = pd.concat([all_explaining_features_df, explaining_features], axis=0)
                continue

            else:
                raise ValueError('unknown SHAP value selection method')

            num_explaining_features = 0
            for j in range(len(shap_values)):
                if shap_values[j] > treshold_val:
                    num_explaining_features += 1
            explaining_features = shap_values_df[i:i + 1].stack().nlargest(num_explaining_features)
            all_explaining_features_df = pd.concat([all_explaining_features_df, explaining_features], axis=0)
        return all_explaining_features_df

### Set Explaining for Features

In [156]:
def set_explaining_features(df_top_err, highest_contributing_features, num_of_features, all_sets_explaining_features, record_idx):
    
    set_explaining_features = []

    for idx_explained_feature in range(num_of_features):
        feature_set = []

        # Iterate over SHAP values to find the highest contributing features
        for idx, row in highest_contributing_features.iterrows():
            if idx[0] == idx_explained_feature:
                feature_set.append((idx[1], row[0]))

        # Get the feature being explained from df_top_err
        explained_feature_index = df_top_err.index[idx_explained_feature]
        feature_set.insert(0, (X_train.columns[explained_feature_index], -1))

        # Append the feature set to the all_sets_explaining_features dictionary for the given record_idx
        if record_idx not in all_sets_explaining_features:
            all_sets_explaining_features[record_idx] = []

        all_sets_explaining_features[record_idx].append(feature_set)

    return all_sets_explaining_features

### Main Function

In [157]:
all_sets_explaining_features = {}

for record_idx in top_anomalies_to_explain:
    
    
    record_to_explain = X_test.loc[record_idx]

    df_err, total_mse = get_errors_df_per_record(record_to_explain)
    num_of_features = get_num_features_with_highest_reconstruction_error(total_mse * df_err.shape[0],
                                                                                      df_err)
    
    df_top_err = df_err.head(num_of_features)
    
    all_sets_explaining_features[record_idx] = []
    
    print("df_top_err")
    print(df_top_err)
    print("record_to_explain")
    print(record_to_explain)
    
    record_to_explain_transposed = record_to_explain.to_frame().T
    
    print("record_to_explain_transposed")
    print(record_to_explain_transposed)
   
    shap_values_all_features = get_shap_values(df_top_err, record_to_explain, num_of_features)
    
    print("shap_values_all_features")
    print(shap_values_all_features)
    print(shap_values_all_features.shape)
    
    
    shap_values_all_features = pd.DataFrame(data=shap_values_all_features, columns=X_train.columns)
    
    print("-----------------------------------")
    
    highest_contributing_features = get_highest_shap_values(shap_values_all_features)
    
    all_sets_explaining_features = set_explaining_features(df_top_err, highest_contributing_features, num_of_features, all_sets_explaining_features, record_idx)
 
    final_set_features = []
    final_set_items = []
    for item in sum(all_sets_explaining_features[record_idx], []):
        if item[0] not in final_set_features:
            final_set_features.append(item[0])
            final_set_items.append(item)
                    
            if return_shap_values:
                all_sets_explaining_features[record_idx] = final_set_items
            else:
                all_sets_explaining_features[record_idx] = final_set_features

        print(all_sets_explaining_features)
        
        
    break

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
df_top_err
   col_name       err
9       V10  0.263231
8        V9  0.232141
0        V1  0.195796
2        V3  0.188821
7        V8  0.165092
21      V22  0.159087
record_to_explain
V1        0.478275
V2        0.993769
V3        0.371772
V4        0.772241
V5        0.644543
V6        0.324007
V7        0.012492
V8        0.367175
V9        0.000000
V10       0.003829
V11       0.488024
V12       0.342055
V13       0.210203
V14       0.654125
V15       0.124266
V16       0.239628
V17       0.354636
V18       0.267213
V19       0.543883
V20       0.691433
V21       0.215637
V22       0.900117
V23       0.738338
V24       0.395484
V25       0.493217
V26       0.426871
V27       0.316695
V28       0.309394
Amount    0.000000
Name: 154587, dtype: float64
record_to_explain_transposed
              V1        V2        V3        V4        V5        V6        V7  \
154587  0.478275  0.993769  0.371772  0.772241  0.644543  0.324007  0.012492   

         

7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
Record
[[0.4782754  0.99376908 0.3717716  0.77224087 0.64454336 0.3240068
  0.01249154 0.36717458 0.         0.00382918 0.48802375 0.34205528
  0.21020302 0.65412481 0.12426603 0.2396281  0.35463601 0.26721275
  0.54388346 0.69143302 0.21563681 0.90011749 0.73833815 0.39548379
  0.49321723 0.42687131 0.31669487 0.30939377 0.        ]]
Counter
21
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
Record
[[4.78275398e-01 7.66490419e-01 8.81364903e-01 ... 4.18976135e-01
  3.12696634e-01 5.82379309e-03]
 [4.78275398e-01 7.70066651e-01 8.40298490e-01 ... 4.16345145e-01
  3.13422663e-01 1.04705276e-04]
 [4.78275398e-01 7.53117667e-01 8.68140819e-01 ... 4.15489266e-01
  3.11911316e-01 1.47389219e-02]
 ...
 [4.78275398e-01 9.93769078e-01 3.71771602e-01 ... 3.16694867e-01
  3.09393773e-01 0.00000000e+00]
 [4.78275398e-01 9.93769078e-01 3.71771602e-01 ... 3.16694867e-01
  3.09393773e-01 0.00000000e+00]
 [4.78275398e-01 9.93769078e-01 3.71771602e-01 ... 3.16694867e-01
  3

NameError: name 'self' is not defined